In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.config import load_config
import yfinance as yf
import numpy as np



class Var:
    def __init__(self,config):
        self.config = config


        
    def load_data(self):
        """
        Loads in data from yfinance
        """
        data = yf.download(tickers=self.config["combined_assets"],start=self.config['start_date'],end=self.config['end_date'])['Close']
        data = data.dropna()
        data.drop_duplicates(inplace=True)
        return data

    
    def get_var(self,ci=0.95):
        """
        value at risk
        """
        # data from data fetcher
        data = self.load_data()
        
        #returns
        returns = data.pct_change().dropna()

        value_at_risk = np.percentile(returns,(1 - ci)*100)
        return value_at_risk
    
    def get_cvar(self,ci=0.95):
        """
        Conditional Value at Risk
        """

        returns = pd.read_csv(self.config['returns_path'],delimiter=",")

        tail_risk = returns[returns < value_at_risk]
        cvar = np.mean(tail_risk)
        return cvar
    
    def plot_returns(self):
        """ plot VaR, CVar and returns """

        # get values
        returns = pd.read_csv(self.config['returns_path'],delimiter=",")
        # var and cvar
        value_at_risk = self.get_var()
        cvar = self.get_cvar()
        
        print(f' Returns: {returns}')
        print(f'Value at Risk: {value_at_risk}')
        print(f'Conditional Value at Risk: {cvar:.4}%')
        plt.figure(figsize=(10, 6))
        plt.hist(returns, bins=100, label="Returns Distribution", alpha=0.7)
        plt.axvline(value_at_risk, color='r', linestyle='dashed', linewidth=2, label=f'VaR (5%): {value_at_risk:.4f}')
        plt.axvline(x=cvar, color='green', linestyle='--', label=f'CVaR ({self.cvar:.4f})')
        plt.title('Distribution of Returns and Value at Risk')
        plt.xlabel('Returns')
        plt.ylabel('Frequency')
        plt.legend()
        plt.grid(True)
        plt.show()

if __name__ == "__main__":
    config = load_config()
    var_config = Var(config)
    var_config.load_data()
    var = var_config.get_var()
    print(f'Value at Risk: {var}')
    cvar = var_config.get_cvar()
    print(f'Conditional Value at Risk: {cvar}')
    var_config.plot_returns()


/tmp/ipykernel_109853/707504999.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers=self.config["combined_assets"],start=self.config['start_date'],end=self.config['end_date'])['Close']
[*****                 10%                       ]  3 of 31 completed